In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import random
import itertools
import numpy as np
import pandas as pd
import random

In [3]:
# T>R>P>S and 2R>T+S
# Typical example: T=5,R=3,P=1,S=0 (but keep it symbolic).

In [4]:
# -----------------------------
# PAYOFF MATRIX
# -----------------------------
# Payoffs for (player, opponent)
# R = Reward (mutual cooperation)
# T = Temptation (you defect, they cooperate)
# P = Punishment (mutual defection)
# S = Sucker (you cooperate, they defect)
R, T, P, S = 3, 5, 1, 0

PAYOFFS = {
    ('C', 'C'): (R, R),
    ('C', 'D'): (S, T),
    ('D', 'C'): (T, S),
    ('D', 'D'): (P, P)
}

In [5]:
# -----------------------------
# BASE STRATEGY CLASS
# -----------------------------
class Strategy:
    def __init__(self, name):
        self.name = name
        self.reset()

    def reset(self):
        self.history = []

    def move(self, opponent_history):
        """Override this in each strategy."""
        raise NotImplementedError


# -----------------------------
# STRATEGY DEFINITIONS
# -----------------------------

class Cooperator(Strategy):
    def __init__(self):
        super().__init__("Cooperator")

    def move(self, opponent_history):
        return 'C'


class Cheater(Strategy):
    def __init__(self):
        super().__init__("Cheater")

    def move(self, opponent_history):
        return 'D'


class Copycat(Strategy):
    def __init__(self):
        super().__init__("Copycat")

    def move(self, opponent_history):
        if not opponent_history:
            return 'C'
        return opponent_history[-1]


class Grudger(Strategy):
    def __init__(self):
        super().__init__("Grudger")
        self.grudge = False

    def reset(self):
        super().reset()
        self.grudge = False

    def move(self, opponent_history):
        if 'D' in opponent_history:
            self.grudge = True
        return 'D' if self.grudge else 'C'


class Detective(Strategy):
    def __init__(self):
        super().__init__("Detective")
        self.probe_moves = ['D', 'C', 'C', 'D']

    def reset(self):
        super().reset()
        self.phase = "probe"

    def move(self, opponent_history):
        # Probe phase (first 4 moves)
        if len(self.history) < 4:
            return self.probe_moves[len(self.history)]
        # After probing
        if 'D' not in opponent_history:
            # Opponent never retaliated, exploit them
            return 'D'
        else:
            # Play like Copycat
            return opponent_history[-1]


class Copykitten(Strategy):
    def __init__(self):
        super().__init__("Copykitten")

    def move(self, opponent_history):
        if len(opponent_history) < 2:
            return 'C'
        # Forgive one accidental defection
        if opponent_history[-1] == 'D' and opponent_history[-2] == 'D':
            return 'D'
        return 'C'

In [6]:
# -----------------------------
# SIMULATION FUNCTION
# -----------------------------
def play_round(p1, p2, noise=0.0):
    """Play one round between two strategies (with optional noise)."""
    m1 = p1.move(p2.history)
    m2 = p2.move(p1.history)

    # Random noise: flip move with probability noise
    if random.random() < noise:
        m1 = 'D' if m1 == 'C' else 'C'
    if random.random() < noise:
        m2 = 'D' if m2 == 'C' else 'C'

    p1.history.append(m1)
    p2.history.append(m2)

    pay1, pay2 = PAYOFFS[(m1, m2)]
    return pay1, pay2


def play_match(p1, p2, rounds=50, noise=0.0):
    """Play repeated rounds and return average payoffs."""
    p1.reset()
    p2.reset()
    total1 = total2 = 0

    for _ in range(rounds):
        pay1, pay2 = play_round(p1, p2, noise)
        total1 += pay1
        total2 += pay2

    avg1 = total1 / rounds
    avg2 = total2 / rounds
    return avg1, avg2


# -----------------------------
# RUN A TOURNAMENT
# -----------------------------
def tournament(strategies, rounds=50, noise=0.0):
    results = pd.DataFrame(
        0.0, index=[s.name for s in strategies], columns=[s.name for s in strategies]
    )
    for s1, s2 in itertools.product(strategies, repeat=2):
        avg1, avg2 = play_match(s1, s2, rounds, noise)
        results.loc[s1.name, s2.name] = avg1
    return results


In [7]:
# -----------------------------
# MAIN
# -----------------------------
if __name__ == "__main__":
    strategies = [
        Cooperator(),
        Cheater(),
        Copycat(),
        Grudger(),
        Detective(),
        Copykitten()
    ]

    results = tournament(strategies, rounds=500, noise=0.05)
    # print("\nAverage Payoff Matrix (rows = player, columns = opponent):\n")
    # print(results.round(2))

In [8]:
results.apply(lambda x: x.round(2))

,Cooperator,Cheater,Copycat,Grudger,Detective,Copykitten
Cooperator,2.97,0.22,2.82,0.38,2.80,2.93
Cheater,4.68,1.05,1.32,1.15,1.33,1.40
Copycat,3.03,1.05,2.06,1.28,2.03,2.95
Grudger,4.71,1.14,1.36,1.16,1.28,1.58
Detective,3.04,1.11,1.92,1.10,2.22,2.98
Copykitten,2.97,1.05,2.88,1.15,2.58,2.93
